# Kézzel írott számjegyek felismerése

In [ ]:
import numpy as np 
import math
import matplotlib.pyplot as plt
%matplotlib inline
import struct
import gc
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

### MNIST adatbázis betöltése

In [ ]:
# tanito bemenetek beolvasasa
f=open("mnist_database/train_60k_images.dat","rb")
signature,=struct.unpack(">I",f.read(4))
numimages,=struct.unpack(">I",f.read(4))
rows,=struct.unpack(">I",f.read(4))
cols,=struct.unpack(">I",f.read(4))
if (rows!=28) or (cols!=28) or (signature!=2051) or (numimages!=60000):
    print("Hiba!")
else:
    train_images=np.empty([60000,784],dtype="float32")
    for i in range(60000):
        train_images[i]=np.array(struct.unpack("784B",f.read(784)),dtype="float32")/255
f.close()

# tanito kimenetek beolvasasa
f=open("mnist_database/train_60k_labels.dat","rb")
fdata=f.read()
f.close()
signature,=struct.unpack(">I",fdata[0:4])
numlabels,=struct.unpack(">I",fdata[4:8])
if (signature!=2049) or (numlabels!=60000):
    print("Hiba!")
else:
    train_labels=np.zeros([60000,10])
    for i in range(60000):
        temp,=struct.unpack("B",fdata[8+i])
        train_labels[i][temp]=1
fdata=[]

# teszt bemenetek beolvasasa
f=open("mnist_database/test_10k_images.dat","rb")
signature,=struct.unpack(">I",f.read(4))
numimages,=struct.unpack(">I",f.read(4))
rows,=struct.unpack(">I",f.read(4))
cols,=struct.unpack(">I",f.read(4))
if (rows!=28) or (cols!=28) or (signature!=2051) or (numimages!=10000):
    print("Hiba!")
else:
    test_images=np.empty([10000,784],dtype="float32")
    for i in range(10000):
        test_images[i]=np.array(struct.unpack("784B",f.read(784)),dtype="float32")/255
f.close()

# teszt kimenetek beolvasasa
f=open("mnist_database/test_10k_labels.dat","rb")
fdata=f.read()
f.close()
signature,=struct.unpack(">I",fdata[0:4])
numlabels,=struct.unpack(">I",fdata[4:8])
if (signature!=2049) or (numlabels!=10000):
    print("Hiba!")
else:
    test_labels=np.zeros([10000,10])
    for i in range(10000):
        temp,=struct.unpack("B",fdata[8+i])
        test_labels[i][temp]=1
fdata=[]

gc.collect()
print("MNIST adatbazis betoltve.")

### Osztályozás egyszerű regresszióval

In [ ]:
# batch_size darab, veletlenszeruen valasztott elemet ad vissza a tanito adatokbol
def get_batch(batch_size):
    global train_images, train_labels
    perm=np.random.permutation(train_images.shape[0])
    perm=perm[:batch_size]
    return [train_images[perm,:],train_labels[perm,:]]

In [ ]:
sess=tf.Session()
# bemenet es kimenet
x =tf.placeholder(tf.float32,shape=[None,train_images.shape[1]])
y_=tf.placeholder(tf.float32,shape=[None,train_labels.shape[1]])
# sulymatrix es biasvektor
W=tf.Variable(tf.zeros([train_images.shape[1],train_labels.shape[1]]))
b=tf.Variable(tf.zeros([train_labels.shape[1]]))
# inicializalas
sess.run(tf.initialize_all_variables())
# regresszio
y=tf.nn.softmax(tf.matmul(x,W)+b)
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
# tanitas: 1000 epoch, 50-es kotegek
for i in range(1000):
    batch=get_batch(50)
    train_step.run(session=sess,feed_dict={x: batch[0], y_: batch[1]})

In [ ]:
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1),tf.argmax(y_,1)),tf.float32))
print("Accuracy:\t%g"%accuracy.eval(session=sess,feed_dict={x: test_images, y_: test_labels}))
y_pred=np.argmax(sess.run(y,feed_dict={x:test_images,y_:test_labels}),1)
y_true=np.argmax(test_labels,1)
print("Precision:\t%g"%precision_score(y_true,y_pred,average="macro"))
print("Recall score:\t%g"%recall_score(y_true,y_pred,average="macro"))
print("F1 score:\t%g"%f1_score(y_true,y_pred,average="macro"))
print("Konfuzios matrix:")
print(confusion_matrix(y_true,y_pred))

### Osztályozás konvolúciós hálóval

In [ ]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape=shape,stddev=0.1,name="weights"))
def bias_variable(shape):
    return tf.Variable(tf.constant(value=0.1,shape=shape))
def conv2d(x,W,name):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME",name=name)
def maxpool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

# konvolucios reteg: 1 28x28-as bemenet, 32 5x5-os szuro, 2x2-es zero padding --> 32 28x28-as kimenet
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])
h_conv1=tf.nn.relu(conv2d(tf.reshape(x,[-1,28,28,1]),W_conv1,"h_conv1")+b_conv1)
# maxpool reteg: 32 db 28x28 --> 32 db 14x14
h_pool1=maxpool_2x2(h_conv1)
# konvolucios reteg: 32 28x28-as bemenet, 64 5x5-os szuro, 2x2-es zero padding --> 64 28x28-as kimenet
W_conv2=weight_variable([5,5,32,64])
b_conv2=bias_variable([64])
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2,"h_conv2")+b_conv2)
# maxpool reteg: 64 db 14x14 --> 64 db 7x7
h_pool2=maxpool_2x2(h_conv2)
# a maxpool reteget kilapitjuk, es raadjuk egy teljesen osszekotott retegre: 3136 --> 2014 (ReLU)
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
h_fc1=tf.nn.relu(tf.matmul(tf.reshape(h_pool2,[-1,7*7*64]),W_fc1)+b_fc1)
# dropout
keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
# kimeneti reteg: 1024 --> 10 (softmax)
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [ ]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch=get_batch(50)
    if (i+1)%100==0:
        print("Step %d, accuracy is %g."%(i+1,
                        accuracy.eval(session=sess,feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})))
    train_step.run(session=sess,feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

In [ ]:
y_pred=np.empty(test_images.shape[0])
for i in range(test_images.shape[0]): # egyszerre is ra lehetne adni, de ahhoz sok memoria kell
    out=sess.run(y_conv,feed_dict={x:  np.atleast_2d(test_images[i]),
                                   y_: np.atleast_2d(test_labels[i]), keep_prob: 1.0})
    y_pred[i]=np.argmax(out,1)
y_true=np.argmax(test_labels,1)
print("Precision:\t%g"%precision_score(y_true,y_pred,average="macro"))
print("Recall score:\t%g"%recall_score(y_true,y_pred,average="macro"))
print("F1 score:\t%g"%f1_score(y_true,y_pred,average="macro"))
print("Konfuzios matrix:")
print(confusion_matrix(y_true,y_pred))

### A konvolúciós rétegek kimenetei

In [ ]:
def showConvLayerOutputs(layer,inputdata):
    units=layer.eval(session=sess,feed_dict={x:np.reshape(inputdata,[1,784],order='F'),keep_prob:1.0})
    no_filters=units.shape[3]
    plt.figure(figsize=(32,35))
    for i in range(0,no_filters):
        plt.subplot(math.ceil(no_filters/6.0),6,i+1)
        plt.title("Filter "+str(i))
        plt.imshow(units[0,:,:,i],interpolation="none",cmap="gray")
def showConvLayerWeights(w):
    weights=w.eval(session=sess)
    no_filters=weights.shape[3]
    plt.figure(figsize=(32,35))
    for i in range(0,no_filters):
        plt.subplot(math.ceil(no_filters/6.0),6,i+1)
        plt.title('Filter '+str(i))
        plt.imshow(weights[:,:,0,i],interpolation="none",cmap="gray")

In [ ]:
testInput=test_images[6]
plt.imshow(np.reshape(testInput,[28,28]),interpolation="nearest",cmap="gray");

In [ ]:
showConvLayerOutputs(h_conv1,testInput)
showConvLayerWeights(W_conv1)

In [ ]:
showConvLayerOutputs(h_conv2,testInput)
showConvLayerWeights(W_conv2)